This code will generate Z- Matrix from smiles

In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from IPython.display import display
from rdkit import Chem
from rdkit.Chem import AllChem
import openbabel as ob
from rmgpy.molecule.converter import toOBMol
import pybel as pyb
import os.path
from IPython.display import display

In [2]:
P1='CCC'
prod1 = Molecule().fromSMILES(P1)
P1

'CCC'

In [3]:
prod1

In [4]:
prod1.multiplicity

1

In [5]:
prod1.symmetryNumber

-1.0

In [6]:

#rdmol,rdInds = prod1.toRDKitMol(removeHs=False,returnMapping=True)
#rdmol


In [7]:
def findInternalRotorInds(mol):
    """
    locates the sets of indices corresponding to every internal rotor
    """
    rotors = []
    for atom1 in mol.vertices:
        for atom2, bond in atom1.edges.items():
            if mol.vertices.index(atom1) < mol.vertices.index(atom2) and (bond.isSingle() or bond.isHydrogenBond()) and not mol.isBondInCycle(bond):
                if len(atom1.edges) > 1 and len(atom2.edges) > 1:
                    rotor = []
                    for atm in atom1.edges.keys():
                        if atm == atom2:
                            continue
                        else:
                            rotor.append(mol.atoms.index(atm))
                            break
                    rotor.append(mol.atoms.index(atom1))
                    rotor.append(mol.atoms.index(atom2))
                    for atm in atom2.edges.keys():
                        if atm == atom1:
                            continue
                        else:
                            rotor.append(mol.atoms.index(atm))
                            break
                    rotors.append(rotor)
    return rotors

In [8]:
atomList = [x.atomType.label for x in prod1.atoms]
IntRotList = findInternalRotorInds(prod1)

Finding lowest energy conformer. from Matt's code
This might not be needed since EStoKTp use monte carlo analasis to find the lowest erengy conformer

In [9]:
def getPossibleConformersRDKit(mol):
    """
    Uses Rdkit to automatically generate set of len(mol.atoms)-3)*6 initial geometries, optimizes
    these geometries using MMFF94s, calculates the energies using MMFF94s
    and converts them back in terms of the RMG atom ordering
    Returns the coordinates and energies
    """
    rdmol,rdInds = mol.toRDKitMol(removeHs=False,returnMapping=True)
    rdIndMap = dict()
    for k,atm in enumerate(mol.atoms):
        ind = rdInds[atm]
        rdIndMap[ind] = k

    #AllChem.EmbedMultipleConfs(rdmol,numConfs=(len(mol.atoms)-3)*60,randomSeed=1,)
    AllChem.EmbedMultipleConfs(rdmol,numConfs=2,randomSeed=1,)
    energies = []
    xyzs = []
    for i in xrange(rdmol.GetNumConformers()):
        v = 1
        while v == 1:
            v = AllChem.MMFFOptimizeMolecule(rdmol,mmffVariant='MMFF94s',confId=i,maxIters=500,ignoreInterfragInteractions=False)
        mp = AllChem.MMFFGetMoleculeProperties(rdmol,mmffVariant='MMFF94s')
        ff = AllChem.MMFFGetMoleculeForceField(rdmol,mp,confId=i)
        E = ff.CalcEnergy()
        energies.append(E)
        cf = rdmol.GetConformer(i)
        xyz = []
        for j in xrange(cf.GetNumAtoms()):
            pt = cf.GetAtomPosition(j)
            xyz.append([pt.x,pt.y,pt.z])
        xyz = [xyz[rdIndMap[i]] for i in xrange(len(xyz))]
        xyzs.append(xyz)
        
    return xyzs,energies

In [10]:
xyzs,es = getPossibleConformersRDKit(prod1)

In [11]:
#if len(IntRotList)>0:
#    xyzs,es = getPossibleConformersRDKit(prod1)

In [12]:
def getMinEnergyConformer(xyzs,energies):
    minval = min(energies)
    minind = energies.index(minval)
    return xyzs[minind]

In [13]:
#if len(IntRotList)>0:
xyzmin = getMinEnergyConformer(xyzs,es)
#else:
#    xyzmin=prod1
#atomList = [x.atomType.label for x in prod1.atoms]
#IntRotList = findInternalRotorInds(prod1)

In [14]:
xyzmin

[[-0.061593640199089736, 0.6032299529080738, -0.12720403753961873],
 [1.248890822837117, -0.056027078996183075, 0.2682431317893901],
 [-1.2024084001940087, -0.399207557478795, -0.17224686444889364],
 [-0.3014158911002869, 1.3968081634462168, 0.5891250338694982],
 [0.04910429656573524, 1.074257362509034, -1.1102032668079778],
 [1.527476179488654, -0.8359102809471514, -0.4477045213213312],
 [1.1750388198482453, -0.5115936571810953, 1.260923615928825],
 [2.0538299296328244, 0.6851395440935312, 0.29359552859107513],
 [-1.3537388552461167, -0.8656231016025093, 0.8065100570243616],
 [-2.133881778228453, 0.09886247231900364, -0.4589197275020419],
 [-1.0013014834046259, -1.1899358190701232, -0.9021189495832999]]

In [15]:
atomList

['Cs', 'Cs', 'Cs', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']

In [16]:
IntRotList

[[3, 0, 1, 2], [0, 1, 2, 8]]

In [17]:
 index = prod1.toInChIKey()
 index

'ATUOYWHBWRKTHZ-UHFFFAOYSA-N'

In [18]:
    cwd = os.getcwd()
    save_path = cwd + '/' + str(index)
    save_path
    #os.makedirs(save_path)
    try:
        os.makedirs(save_path)
    except OSError as ex:
        if ex.errno == 17 and os.path.isdir(save_path):
            pass
        else:
            print('An error happened trying to create ' + save_path)
            raise 

In [19]:
def writecartesian(atomList,xyzmin,introtlist,index):
    cwd = os.getcwd()
    save_path = cwd + '/' + str(index)
    save_path
    #os.makedirs(save_path)
    try:
        os.makedirs(save_path)
    except OSError as ex:
        if ex.errno == 17 and os.path.isdir(save_path):
            pass
        else:
            print('An error happened trying to create ' + save_path)
            raise 
    filename = os.path.join(save_path, str(index) + '_xyz.gjf')
    #write to a gaussian input file 
    file = open(filename, 'wb')
    file.write('#P\n\n')
    file.write(index + '\n\n')
    file.write('0 1\n')
    for i in range(len(xyzmin)):
        file.write(' ' + atomList[i][0] + '            '+ str(xyzmin[i][0]) + ' '+ str(xyzmin[i][1]) + ' '+ str(xyzmin[i][2]) + '\n')
        file.write('\n')
    file.close()
  

In [20]:
writecartesian(atomList,xyzmin,IntRotList,index)

In [21]:
cartesian=[]
for i in range(len(xyzmin)):
    cartesian.append([atomList[i][0],np.array([xyzmin[i][0], xyzmin[i][1], xyzmin[i][2]],dtype='f8')])
cartesian

[['C', array([-0.06159364,  0.60322995, -0.12720404])],
 ['C', array([ 1.24889082, -0.05602708,  0.26824313])],
 ['C', array([-1.2024084 , -0.39920756, -0.17224686])],
 ['H', array([-0.30141589,  1.39680816,  0.58912503])],
 ['H', array([ 0.0491043 ,  1.07425736, -1.11020327])],
 ['H', array([ 1.52747618, -0.83591028, -0.44770452])],
 ['H', array([ 1.17503882, -0.51159366,  1.26092362])],
 ['H', array([ 2.05382993,  0.68513954,  0.29359553])],
 ['H', array([-1.35373886, -0.8656231 ,  0.80651006])],
 ['H', array([-2.13388178,  0.09886247, -0.45891973])],
 ['H', array([-1.00130148, -1.18993582, -0.90211895])]]

In [22]:
len(cartesian)

11

write roters

In [23]:
def writeroters(introtlist):
 for i in range(len(introtlist)):
        print('D    '+ str(introtlist[i][0] + 1) + '   ' + str(introtlist[i][1] + 1) + '   ' + str(introtlist[i][2] + 1) + '   ' + str(introtlist[i][3] + 1) + '   S   36   10.00')

In [24]:
writeroters(IntRotList)

D    4   1   2   3   S   36   10.00
D    1   2   3   9   S   36   10.00


converting the  cartesian geomatry into Zmatrix
the algorithm and code  is taken from 
https://github.com/jevandezande/zmatrix

In [25]:
 def add_first_three_to_zmatrix(cartesian):
        """The first three atoms need to be treated differently"""
        # First atom
        name, position= cartesian[0]
        zmatrix.append([name, [[], [], []]])

        # Second atom
        if len(cartesian) > 1:
            name, position = cartesian[1]
            atom1 = cartesian[0]
            pos1 = atom1[1]
            q = pos1 - position
            distance = np.sqrt(np.dot(q, q))
            zmatrix.append([name, [[0, distance], [], []]])

        # Third atom
        if len(cartesian) > 2:
            name, position = cartesian[2]
            atom1, atom2 = cartesian[:2]
            pos1, pos2 = atom1[1], atom2[1]
            q = pos1 - position
            r = pos2 - pos1
            q_u = q / np.sqrt(np.dot(q, q))
            r_u = r / np.sqrt(np.dot(r, r))
            distance = np.sqrt(np.dot(q, q))
            # Angle between a and b = acos(dot(a, b)) / (|a| |b|))
            angle = np.arccos(np.dot(-q_u, r_u))
            zmatrix.append(
                [name, [[0, distance], [1, np.degrees(angle)], []]])
        print(zmatrix)

In [26]:
def add_atom_to_zmatrix(cartesian, i, line):
        """Generates an atom for the zmatrix
        (assumes that three previous atoms have been placed in the cartesian coordiantes)"""
        name, position = line
        atom1, atom2, atom3 = cartesian[:3]
        pos1, pos2, pos3 = atom1[1], atom2[1], atom3[1]
        # Create vectors pointing from one atom to the next
        q = pos1 - position
        r = pos2 - pos1
        s = pos3 - pos2
        position_u = position / np.sqrt(np.dot(position, position))
        # Create unit vectors
        q_u = q / np.sqrt(np.dot(q, q))
        r_u = r / np.sqrt(np.dot(r, r))
        s_u = s / np.sqrt(np.dot(s, s))
        distance = np.sqrt(np.dot(q, q))
        # Angle between a and b = acos(dot(a, b)) / (|a| |b|))
        angle = np.arccos(np.dot(-q_u, r_u))
        angle_123 = np.arccos(np.dot(-r_u, s_u))
        # Dihedral angle =
        #   acos(dot(normal_vec1, normal_vec2)) / (|normal_vec1| |normal_vec2|))
        plane1 = np.cross(q, r)
        plane2 = np.cross(r, s)
        dihedral = np.arccos(np.dot(
            plane1, plane2) / (np.sqrt(np.dot(plane1, plane1)) * np.sqrt(np.dot(plane2, plane2))))
        # Convert to signed dihedral angle
        if np.dot(np.cross(plane1, plane2), r_u) < 0:
            dihedral = -dihedral

        coords = [[0, distance], [1, np.degrees(angle)], [
            2, np.degrees(dihedral)]]
        atom = [name, coords]
        zmatrix.append(atom)

In [27]:
def cartesian_to_zmatrix(cartesian):
        """Convert the cartesian coordinates to a zmatrix"""
        add_first_three_to_zmatrix(cartesian)
        for i, atom in enumerate(cartesian[3:], start=3):
            add_atom_to_zmatrix(cartesian,i, atom)

        return zmatrix

In [28]:
zmatrix = []
cartesian_to_zmatrix(cartesian)

[['C', [[], [], []]], ['C', [[0, 1.5193313745542252], [], []]], ['C', [[0, 1.5193314763766552], [1, 111.65877337306692], []]]]


[['C', [[], [], []]],
 ['C', [[0, 1.5193313745542252], [], []]],
 ['C', [[0, 1.5193314763766552], [1, 111.65877337306692], []]],
 ['H',
  [[0, 1.0956315196333508], [1, 109.44600559496776], [2, 121.33784244924358]]],
 ['H',
  [[0, 1.0956314793494972],
   [1, 109.44598526560351],
   [2, -121.33778502439287]]],
 ['H',
  [[0, 2.1677149408184699],
   [1, 28.129209182950127],
   [2, -60.244185516753284]]],
 ['H',
  [[0, 2.1677153653509906], [1, 28.129213142047924], [2, 60.244318643962153]]],
 ['H',
  [[0, 2.1584249664476056],
   [1, 28.401929847018113],
   [2, -179.99993078638664]]],
 ['H',
  [[0, 2.1677155170597069], [1, 96.200987026832038], [2, 24.312137794520734]]],
 ['H',
  [[0, 2.1584253476690405],
   [1, 140.06069434309703],
   [2, 3.1933873632997163e-05]]],
 ['H',
  [[0, 2.1677147961122167],
   [1, 96.201047974469532],
   [2, -24.312185404384746]]]]

In [29]:
zmatrix

[['C', [[], [], []]],
 ['C', [[0, 1.5193313745542252], [], []]],
 ['C', [[0, 1.5193314763766552], [1, 111.65877337306692], []]],
 ['H',
  [[0, 1.0956315196333508], [1, 109.44600559496776], [2, 121.33784244924358]]],
 ['H',
  [[0, 1.0956314793494972],
   [1, 109.44598526560351],
   [2, -121.33778502439287]]],
 ['H',
  [[0, 2.1677149408184699],
   [1, 28.129209182950127],
   [2, -60.244185516753284]]],
 ['H',
  [[0, 2.1677153653509906], [1, 28.129213142047924], [2, 60.244318643962153]]],
 ['H',
  [[0, 2.1584249664476056],
   [1, 28.401929847018113],
   [2, -179.99993078638664]]],
 ['H',
  [[0, 2.1677155170597069], [1, 96.200987026832038], [2, 24.312137794520734]]],
 ['H',
  [[0, 2.1584253476690405],
   [1, 140.06069434309703],
   [2, 3.1933873632997163e-05]]],
 ['H',
  [[0, 2.1677147961122167],
   [1, 96.201047974469532],
   [2, -24.312185404384746]]]]

In [30]:
len(zmatrix)

11

In [31]:
zmatrix[3][1]
pos1, pos2, pos3 = zmatrix[3][1]
pos2
pos2[0]+1
pos3[1]
format(pos3[1],'.4f')

'121.3378'

In [32]:
def writezmatrix(zmatrix):
    cwd = os.getcwd()
    save_path = cwd + '/' + str(index)
    save_path
    #os.makedirs(save_path)
    try:
        os.makedirs(save_path)
    except OSError as ex:
        if ex.errno == 17 and os.path.isdir(save_path):
            pass
        else:
            print('An error happened trying to create ' + save_path)
            raise 
    filename = os.path.join(save_path, str(index) + '_zmat.gjf')
    #write to a gaussian input file 
    file = open(filename, 'wb')
    file.write('#P\n\n')
    file.write(index + '\n\n')
    file.write('0 1\n')
    for i in range(len(zmatrix)):
        pos1, pos2, pos3 = zmatrix[i][1]
        if i == 0:
            file.write(zmatrix[i][0]) 
        if i == 1:
            file.write(zmatrix[i][0] + 
                       ' '+str(pos1[0]+1)+ ' ' + str(format(pos1[1],'.4f')))   
        if i == 2:
            file.write(zmatrix[i][0] + 
                       ' '+str(pos1[0]+1)+ ' ' + str(format(pos1[1],'.4f')) + 
                       ' '+str(pos2[0]+1)+ ' ' + str(format(pos2[1],'.4f')) )         
        if i > 2:
            file.write(zmatrix[i][0] + 
                       ' '+str(pos1[0]+1)+ ' ' + str(format(pos1[1],'.4f')) + 
                       ' '+str(pos2[0]+1)+ ' ' + str(format(pos2[1],'.4f')) +
                       ' '+str(pos3[0]+1)+ ' ' + str(format(pos3[1],'.4f')) ) 
        #file.write( zmatrix[i][0] + ' '+ str(zmatrix[i][1]) + '\n')
        file.write('\n')
    file.close()

In [33]:
writezmatrix(zmatrix)

In [34]:
writeroters(IntRotList)

D    4   1   2   3   S   36   10.00
D    1   2   3   9   S   36   10.00


In [35]:
zmatrix

[['C', [[], [], []]],
 ['C', [[0, 1.5193313745542252], [], []]],
 ['C', [[0, 1.5193314763766552], [1, 111.65877337306692], []]],
 ['H',
  [[0, 1.0956315196333508], [1, 109.44600559496776], [2, 121.33784244924358]]],
 ['H',
  [[0, 1.0956314793494972],
   [1, 109.44598526560351],
   [2, -121.33778502439287]]],
 ['H',
  [[0, 2.1677149408184699],
   [1, 28.129209182950127],
   [2, -60.244185516753284]]],
 ['H',
  [[0, 2.1677153653509906], [1, 28.129213142047924], [2, 60.244318643962153]]],
 ['H',
  [[0, 2.1584249664476056],
   [1, 28.401929847018113],
   [2, -179.99993078638664]]],
 ['H',
  [[0, 2.1677155170597069], [1, 96.200987026832038], [2, 24.312137794520734]]],
 ['H',
  [[0, 2.1584253476690405],
   [1, 140.06069434309703],
   [2, 3.1933873632997163e-05]]],
 ['H',
  [[0, 2.1677147961122167],
   [1, 96.201047974469532],
   [2, -24.312185404384746]]]]

In [36]:
i=4
pos1, pos2, pos3 = zmatrix[i][1]
#print(zmatrix[i][0],pos1, pos2, pos3)
#print(pos1[0],pos1[1])
atom1=int(pos1[0])+1
atom2=int(pos2[0])+1
#print(atom1)
#print(zmatrix[i-atom1][0])
#print('r'+str(zmatrix[i-atom1][0])+str(zmatrix[i][0])+str(atom1))
distant=[]
distant=[str('r'+str(zmatrix[atom1][0])+str(zmatrix[i][0])+str(i+1)),format(pos1[1],'.4f')]
#print('distant',distant)
#print('a'+str(zmatrix[i-atom1][0])+str(zmatrix[i-atom2][0])+str(zmatrix[i][0])+str(atom1))
angle=[]
angle=[str('a'+str(zmatrix[atom1][0])+str(zmatrix[atom2][0])+str(zmatrix[i][0])+str(i+1))
       ,format(pos2[1],'.4f')]
#print('angle',angle)
#print(str('d'+str(i)))
dihidral=[]
dihidral=[str('d'+str(i)),format(pos3[1],'.4f')]
print(distant,angle,dihidral)

(['rCH5', '1.0956'], ['aCCH5', '109.4460'], ['d4', '-121.3378'])


In [37]:
varibables=[]

In [38]:
def writezmatrix2(zmatrix,fname):
    cwd = os.getcwd()
    save_path = cwd + '/' + str(index)
    save_path
    #os.makedirs(save_path)
    try:
        os.makedirs(save_path)
    except OSError as ex:
        if ex.errno == 17 and os.path.isdir(save_path):
            pass
        else:
            print('An error happened trying to create ' + save_path)
            raise 
    filename = os.path.join(save_path, str(index) + fname) #'_var_zmat.gjf')
    #write to a gaussian input file 
    file = open(filename, 'a+')
    #file.write('#P\n\n')
    #file.write(index + '\n\n')
    #file.write('0 1\n')
    for i in range(len(zmatrix)):
        pos1, pos2, pos3 = zmatrix[i][1]

        if i == 0:
            file.write(zmatrix[i][0]) 
        if i == 1:
            atom1=int(pos1[0])+1
            #atom2=int(pos2[0])+1
            distant=[]
            distant=[str('r'+str(zmatrix[atom1][0])+str(atom1)+str(zmatrix[i][0])+str(i+1)),format(pos1[1],'.4f')]
            varibables.append(distant)
            #angle=[]
            #angle=[str('a'+str(zmatrix[atom1][0])+str(atom1)+str(zmatrix[atom2][0])+str(atom2)+str(zmatrix[i][0])+str(i+1))
            #   ,format(pos2[1],'.4f')]
            #varibables.append(angle)
            file.write(zmatrix[i][0] + 
                       ' '+str(pos1[0]+1)+ ' ' + str(distant[0]))    
        if i == 2:
            atom1=int(pos1[0])+1
            atom2=int(pos2[0])+1
            distant=[]
            distant=[str('r'+str(zmatrix[atom1][0])+str(atom1)+str(zmatrix[i][0])+str(i+1)),format(pos1[1],'.4f')]
            varibables.append(distant)
            angle=[]
            angle=[str('a'+str(zmatrix[atom1][0])+str(atom1)+str(zmatrix[atom2][0])+str(atom2)+str(zmatrix[i][0])+str(i+1))
               ,format(pos2[1],'.4f')]
            varibables.append(angle)
            file.write(zmatrix[i][0] + 
                       ' '+str(pos1[0]+1)+ ' ' + str(distant[0]) + 
                       ' '+str(pos2[0]+1)+ ' ' + str(angle[0]))    
        if i > 2:
            atom1=int(pos1[0])+1
            atom2=int(pos2[0])+1
            distant=[]
            distant=[str('r'+str(zmatrix[atom1][0])+str(atom1)+str(zmatrix[i][0])+str(i+1)),format(pos1[1],'.4f')]
            varibables.append(distant)
            angle=[]
            angle=[str('a'+str(zmatrix[atom1][0])+str(atom1)+str(zmatrix[atom2][0])+str(atom2)+str(zmatrix[i][0])+str(i+1))
               ,format(pos2[1],'.4f')]
            varibables.append(angle)
            dihidral=[]
            dihidral=[str('d'+str(i+1)),format(pos3[1],'.4f')]
            varibables.append(dihidral)
            file.write(zmatrix[i][0] + 
                       ' '+str(pos1[0]+1)+ ' ' + str(distant[0]) + 
                       ' '+str(pos2[0]+1)+ ' ' + str(angle[0]) +
                       ' '+str(pos3[0]+1)+ ' ' + str(dihidral[0]) ) 
        #file.write( zmatrix[i][0] + ' '+ str(zmatrix[i][1]) + '\n')
        file.write('\n')
    #reomove ! in the write statement below before writing estoktp this so it will  oen in gaussian
    file.write('\nintcoor\n')
    for j in range(len(varibables)):
        file.write(varibables[j][0] + '\t\t' + varibables[j][1] + '\n')
            
    file.close()

In [39]:
#writezmatrix2(zmatrix,'_var_zmat.gjf')

In [40]:
varibables

[]

In [41]:
len(IntRotList)

2

In [42]:
IntRotList

[[3, 0, 1, 2], [0, 1, 2, 8]]

In [43]:
def writeestoketp(introtlist,mol,zmatrix,fname):
    cwd = os.getcwd()
    save_path = cwd + '/' + str(index)
    save_path
    #os.makedirs(save_path)
    try:
        os.makedirs(save_path)
    except OSError as ex:
        if ex.errno == 17 and os.path.isdir(save_path):
            pass
        else:
            print('An error happened trying to create ' + save_path)
            raise 
    filename = os.path.join(save_path, str(index) + fname)
    #write to a gaussian input file 
    file = open(filename, 'wb')
    
    file.write('nosmp dthresh ethresh\n')
    file.write('5 1.0 0.00001\n\n')
    file.write('ntau number of sampled coordinates\n')
    file.write(str(len(IntRotList))+'\n')
    file.write('->namehind,hindmn,hindmx,nhindsteps\n')
    for i in range(len(introtlist)):
        file.write(str('d'+str(introtlist[i][3]+1) + ' 0' + ' 120.0'+'\n'))
    
    file.write('\n')
    file.write('nhind\n'+str(len(IntRotList))+'\n\n')
    file.write('-->namehind,hindmn,hindmx,nhindsteps\n')
    for i in range(len(introtlist)):
        file.write(str('d'+str(introtlist[i][3]+1)) + ' 0' + ' 120.0'+ ' 4' + ' 3'+'\n')
    
    file.write('\n')
    file.write('natom natomt ilin\n')
    file.write(str(len(zmatrix)) + ' ' +str(len(zmatrix)) + ' 0'+'\n')
    
    file.write('\n')               
    file.write('charge spin atomlabel\n')
    file.write('0 '+str(mol.multiplicity)+'\n')
    file.close()
    writezmatrix2(zmatrix,fname)
    file = open(filename, 'a+')
    file.write( '\nSymmetryFactor\n 1.\n\nelec\n1\n 0. 1.\n' )
    file.write('end\n')
    file.write('\n')
    file.close()

In [44]:
writeestoketp(IntRotList,prod1,zmatrix,'.dat')